In [2]:
from ultralytics import YOLO
from ultralytics.solutions import distance_calculation
import cv2

In [3]:
model = YOLO('./runs/detect/yolov8m_climbing_crux_model6/weights/best.pt')
names = model.model.names

In [4]:
img = cv2.imread('./../datasets/images/test/23.jpeg')

In [5]:
height, width, _ = img.shape

In [6]:
print('Height:', height)
print('Width:', width)

Height: 1024
Width: 768


In [ ]:
box1 = {}
box2 = {}

In [1]:
dist_obj = distance_calculation.DistanceCalculation()
dist_obj.set_args(names=names, view_img=False)

tracks = model.track(img, persist=True, show=False)
distance = dist_obj.calculate_distance(box1, box2)

NameError: name 'distance_calculation' is not defined